此模型基于大一所学，代码为开源代码

In [5]:
import torch
import torchaudio
from torch.utils.data import DataLoader
from pypinyin import lazy_pinyin
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, file_list):
        # 读取文件列表并解析
        self.file_list = file_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        # 获取音频文件路径和对应的文本
        file_path, text = self.file_list[index]
        
        # 将中文文本转换为拼音
        pinyin_text = " ".join(lazy_pinyin(text))
        
        # 加载音频文件，提取 Mel 频谱
        mel_spec = self.load_mel_spec(file_path)
        
        return mel_spec, pinyin_text

    def load_mel_spec(self, file_path):
        # 读取音频并提取 Mel 频谱
        waveform, sample_rate = torchaudio.load(file_path)
        mel_spec = self.compute_mel_spectrogram(waveform, sample_rate)
        return mel_spec

    def compute_mel_spectrogram(self, waveform, sample_rate):
        # 使用 torchaudio 提取 Mel 频谱
        mel_transform = torchaudio.transforms.MelSpectrogram(sample_rate)
        mel_spec = mel_transform(waveform)
        return mel_spec


In [8]:
import torchaudio
from torchaudio.pipelines import TACOTRON2_WAVERNN_PHONE_LJSPEECH
from torch.utils.data import DataLoader

# 加载预训练的 Tacotron 2 模型
tacotron2 = TACOTRON2_WAVERNN_PHONE_LJSPEECH.get_text_to_spectrogram()
wavernn = TACOTRON2_WAVERNN_PHONE_LJSPEECH.get_vocoder()

# 假设你有自己的数据集
dataset = MyDataset()  # 这里要定义自己的数据集类
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


TypeError: MyDataset.__init__() missing 1 required positional argument: 'file_list'

In [6]:
import torch
from torch import nn
import torch.optim as optim

# 设定较小的学习率
optimizer = optim.Adam(tacotron2.parameters(), lr=1e-5)
loss_fn = nn.MSELoss()

# 微调 Tacotron 2 和 WaveRNN
for epoch in range(10):  # 训练 10 个 epoch
    for mel_spec, text in dataloader:
        optimizer.zero_grad()
        
        # 使用模型生成 Mel 频谱
        mel_spec_pred, _ = tacotron2(text)
        
        # 计算损失并反向传播
        loss = loss_fn(mel_spec_pred, mel_spec)
        loss.backward()
        optimizer.step()


AttributeError: 'tuple' object has no attribute 'shape'

In [5]:
# 假设你已经完成了训练
text = "zhe shi yi ge ce shi"  # 输入的文本（拼音）
mel_spec, _ = tacotron2.infer(text)  # 生成 Mel 频谱

# 使用 WaveNet vocoder 生成音频波形
waveform = wavernn(mel_spec)

# 保存或播放生成的音频
torchaudio.save("output.wav", waveform, sample_rate=22050)


NameError: name 'mel_spec' is not defined

In [1]:
appid = '20241112002201004'
appkey = 'rRqX2uzmWqLqplMV_M6c'


client = ZhipuAI(api_key="b69f0e809e6601a4ca5c9d9407b8ce2b.Nwpl24uMiaVFZ6Im") 